Gpickle

In [1]:
import pickle
import torch

path = "/home/tommy/Project/PcodeBERT/outputs/data/GNN/gpickle_merged_adjusted_filtered_temp/00/00a6f39a8f7b14f223fa51a9a23aa110112a524799e910e321b162847a875593.gpickle"

with open(path, 'rb') as f:
    data = pickle.load(f)

print(data.keys())

for node_id, embedding in data['node_embeddings'].items():
    print(f"Node ID: {node_id}")
    print(f"Embedding: {embedding}")

dict_keys(['file_path', 'node_embeddings', 'node_sentences', 'num_nodes', 'embedding_dim'])
Node ID: 0x80d4L
Embedding: [ 0.28371355  0.52194285  1.427035    1.108531   -1.4934052  -0.29788998
  0.85302216 -0.18502411 -0.635246    0.5240503   0.48345852  0.9774383
 -0.57075125  1.0520943   0.4785567  -0.32792917  0.4107004  -0.58917993
  0.409956   -0.89654166 -1.5035031  -1.2189919  -0.7839159   0.52976257
  1.2047058  -0.00597998 -0.44528067 -0.73862696 -1.4315134   0.38973242
 -1.284287   -0.6673771  -0.3780485   0.7519026   1.2481289  -1.2328548
 -0.7940837   1.7809712   1.116726    0.36598933 -0.90505725  1.5435379
  0.7722783   0.57501036  1.0607386  -0.553261   -0.11781508 -1.4537977
 -0.9425192  -0.08249355 -0.931706   -0.48095185  1.1757047  -0.03105114
  0.2371872   0.04160293 -1.6158187  -0.5002339   0.39051756  0.63757676
  0.86883205  0.5577863   1.08265    -0.7074783   0.7138338   0.4382918
  0.6617791  -1.3564261  -0.5380895  -0.63841045 -0.8680382   0.82552755
  0.08631

In [1]:
import pickle
import torch

path = "/home/tommy/Project/PcodeBERT/outputs/models/GNN/embeddings_epoch_25/12/12a039af6cec3e8d478f1ce45300e1ce962f2273abd07d0d1ed2f6ea6c60a4cb.gpickle"

with open(path, 'rb') as f:
    data = pickle.load(f)

print(data.keys())

for node_id, embedding in data['node_embeddings'].items():
    print(f"Node ID: {node_id}")
    print(f"Embedding: {embedding}")

dict_keys(['file_path', 'node_embeddings', 'node_sentences', 'num_nodes', 'embedding_dim'])
Node ID: 0x80480b4L
Embedding: [ 5.45289934e-01  1.49051130e+00  1.12286913e+00  1.87074140e-01
 -2.69640386e-01 -2.58157402e-01  2.05827785e+00 -9.16803718e-01
 -2.18580216e-01 -2.27787867e-01  1.24307692e+00  9.07763779e-01
 -6.13447964e-01 -1.42382160e-01  1.11520135e+00 -7.27626383e-01
  1.10948339e-01  2.49875616e-02 -3.54136169e-01 -1.29691386e+00
  1.70173496e-01 -1.25711393e+00 -8.27512205e-01 -7.76802748e-02
 -3.93343419e-02  2.44090352e-02  8.31930995e-01 -1.13262451e+00
 -1.25155735e+00 -5.99901490e-02 -1.32513165e+00  5.49335182e-01
 -1.47622675e-01  5.90746582e-01  4.03052539e-01 -1.88444570e-01
  1.07727361e+00  1.63621628e+00  9.04676557e-01  4.23272431e-01
 -1.45542979e+00  1.89447701e+00 -1.84966713e-01  1.68282732e-01
  5.74908964e-02  3.42926830e-01 -2.58711308e-01 -1.65813839e+00
  5.27424932e-01 -3.87280554e-01  5.43929875e-01  7.44149566e-01
 -7.69056320e-01 -1.50905573e+00

Embedding

In [ ]:
import os
import sys
import pickle
import torch
import numpy as np
from transformers import RobertaForMaskedLM, AutoTokenizer

def load_pretrained_model():
    """載入預訓練的模型和tokenizer"""
    model_path = "/home/tommy/Project/PcodeBERT/outputs/models/pretrain"
    
    print(f"Loading model from: {model_path}")
    
    # 載入tokenizer和model
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = RobertaForMaskedLM.from_pretrained(model_path)
    
    # 設定device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    print(f"Model loaded successfully on device: {device}")
    return model, tokenizer, device

def get_sentence_embedding(sentence, model, tokenizer, device):
    """對單個sentence生成embedding"""
    # Tokenize
    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # 生成embedding
    with torch.no_grad():
        outputs = model.roberta(**inputs)
        # 使用[CLS] token的embedding
        embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
    
    return embedding[0]  # 返回一維array

def process_graph_data(graph_path):
    """處理graph資料並生成embeddings"""
    # 載入模型
    model, tokenizer, device = load_pretrained_model()
    
    # 載入graph資料
    print(f"Loading graph data from: {graph_path}")
    with open(graph_path, 'rb') as f:
        graph = pickle.load(f)
    
    print(f"Graph type: {type(graph)}")
    print(f"Number of nodes: {graph.number_of_nodes()}")
    
    # 處理每個節點
    node_embeddings = {}
    sentences = []
    
    for node_id, node_data in graph.nodes(data=True):
        sentence = node_data.get('sentence', '')
        if sentence:
            print(f"Processing node {node_id}: {sentence[:50]}...")
            
            # 生成embedding
            embedding = get_sentence_embedding(sentence, model, tokenizer, device)
            node_embeddings[node_id] = embedding
            sentences.append(sentence)
    
    print(f"Generated embeddings for {len(node_embeddings)} nodes")
    print(f"Embedding dimension: {len(list(node_embeddings.values())[0])}")
    
    return node_embeddings, sentences

def main():
    # 測試資料路徑
    graph_path = "/home/tommy/Project/PcodeBERT/outputs/gpickle/00/00a3a8743be45f6c561c08be96b325c9d6c0a8b619dc67a4e44c5423bcde5532.gpickle"
    
    try:
        # 處理資料
        node_embeddings, sentences = process_graph_data(graph_path)
        
        # 顯示結果
        print("\n=== Embedding Results ===")
        for i, (node_id, embedding) in enumerate(list(node_embeddings.items())[:3]):  # 只顯示前3個
            print(f"Node {node_id}:")
            print(f"  Sentence: {sentences[i][:100]}...")
            print(f"  Embedding shape: {embedding.shape}")
            print(f"  Embedding mean: {np.mean(embedding):.4f}")
            print(f"  Embedding std: {np.std(embedding):.4f}")
            print()
        
        # 儲存結果
        output_path = "/home/tommy/Project/PcodeBERT/outputs/embeddings/graph_embeddings.pkl"
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        
        result = {
            'node_embeddings': node_embeddings,
            'sentences': sentences,
            'embedding_dim': len(list(node_embeddings.values())[0])
        }
        
        with open(output_path, 'wb') as f:
            pickle.dump(result, f)
        
        print(f"Results saved to: {output_path}")
        
    except Exception as e:
        print(f"Error: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    main()


In [1]:
import os
import sys
import pickle
import torch
import numpy as np
from transformers import RobertaForMaskedLM, AutoTokenizer

def load_pretrained_model():
    """載入預訓練的模型和tokenizer"""
    model_path = "/home/tommy/Project/PcodeBERT/outputs/models/pretrain"
    
    print(f"Loading model from: {model_path}")
    
    # 載入tokenizer和model
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = RobertaForMaskedLM.from_pretrained(model_path)
    
    # 設定device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    
    print(f"Model loaded successfully on device: {device}")
    return model, tokenizer, device

graph_path = "/home/tommy/Project/PcodeBERT/outputs/data/GNN/gpickle_merged_adjusted_filtered_adapter/00/00a6f39a8f7b14f223fa51a9a23aa110112a524799e910e321b162847a875593.gpickle"

with open(graph_path, 'rb') as f:
    data = pickle.load(f)

sample_sentence = data.nodes["0x10002c24L"].get('sentence', '')
print(f"Sample sentence: {sample_sentence}")


model, tokenizer, device = load_pretrained_model()

inputs = tokenizer(sample_sentence, return_tensors="pt", truncation=True, padding=True, max_length=512)
inputs = {k: v.to(device) for k, v in inputs.items()}
with torch.no_grad():
    outputs = model.roberta(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].cpu().numpy()
print(f"Embedding shape: {embedding.shape}")

/home/tommy/miniconda3/envs/PcodeBERT/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: 'dict' object has no attribute 'nodes'

Model

In [1]:
import torch

file_path = "/home/tommy/Project/PcodeBERT/outputs/adapter/adapter_model.pt"

adapter_state = torch.load(file_path, map_location='cpu')
print(adapter_state.keys())

odict_keys(['mapper.0.weight', 'mapper.0.bias', 'mapper.2.weight', 'mapper.2.bias', 'mapper.3.weight', 'mapper.3.bias'])


In [ ]:
import pickle

file = "/home/tommy/Project/PcodeBERT/outputs/models/GNN/embeddings_epoch_25/00/00a6f39a8f7b14f223fa51a9a23aa110112a524799e910e321b162847a875593.gpickle"

with open(file, 'rb') as f:
    data = pickle.load(f)

for